---
<center><h1>Chatbot</h1></center>

---

<p>Dans la première séance, nous avons évoqué les chatbots comme l'une des principales applications réelles
du traitement automatique des langues. A présent, nous en savons suffisamment pour créer un chatbot
basique qui pourrait être “entraîné” à l'aide d'un corpus prédéfini et fournir des réponses à des requêtes
en utilisant des concepts de similarité. Dans ce tp, il est demandé de développer un chatbot en utilisant
les concepts de vectorisation et la similarité cosinus.</p>

<p>La condition la plus importante pour construire un chatbot est relatif au corpus ou les données textuelles
sur lesquelles le chatbot sera entraîné. Le corpus doit être pertinent et exhaustif. Si vous construisez un
chatbot pour le département des ressources humaines (RH) de votre organisation, vous aurez
généralement besoin d'un corpus contenant toutes les politiques RH pour entraîner le chatbot et non d'un
corpus contenant des discours présidentiels. Vous devrez également vous assurer que le temps de réponse
est acceptable et que le robot ne prend pas un temps excessif pour répondre. Idéalement, le chatbot devrait
aussi ressembler à un être humain et avoir un taux de précision acceptable.</p>

<p>Pour les besoins du chatbot que vous allez développer, vous utiliserez une base de données de questions
et de réponses recueillies sur le site Web d'Amazon pour diverses catégories de produits
(http://jmcauley.ucsd.edu/data/amazon/qa/). Dans un premier temps, vous vous focalisez uniquement
aux données relatives aux produits électroniques sous format json
(http://jmcauley.ucsd.edu/data/amazon/qa/qa_Electronics.json.gz).</p>
<p>Extrait des cinq premières lignes du fichier json:</p>

```json
{"questionType": "yes/no", "asin": "0594033926", "answerTime": "Dec 27, 2013", "unixTime": 1388131200,
"question": "Is this cover the one that fits the old nook color? Which I believe is 8x5.", "answerType": "Y",
"answer": "Yes this fits both the nook color and the same-shaped nook tablet"}
```
```json
{"questionType": "yes/no", "asin": "0594033926", "answerTime": "Jan 5, 2015", "unixTime": 1420444800,
"question": "Does it fit Nook GlowLight?", "answerType": "N", "answer": "No. The nook color or color
tablet"}
```
```json
{"answer": "I don't think so. The nook color is 5 x 8 so not sure anything smaller would stay locked in,
but would be close.", "asin": "0594033926", "answerTime": "2 days ago", "question": "Would it fit Nook 1st
Edition? 4.9in x 7.7in ?", "questionType": "open-ended"}
```
```json
{"questionType": "yes/no", "asin": "0594033926", "answerTime": "17 days ago", "question": "Will this fit a
Nook Color that's 5 x 8?", "answerType": "Y", "answer": "yes"}
```
```json
{"questionType": "yes/no", "asin": "0594033926", "answerTime": "Feb 10, 2015", "unixTime": 1423555200,
"question": "will this fit the Samsung Galaxy Tab 4 Nook 10.1", "answerType": "N", "answer": "No, the tab
is smaller than the 'color'"}
```

<p>Comme nous pouvons le voir, chaque ligne de données est au format d'un dictionnaire avec diverses
paires clé-valeur.</p>

<i>Voici les étapes principales pour concevoir un chatbot</i>

1. Stocker toutes les questions du corpus dans une liste

2. Stocker toutes les réponses correspondantes du corpus dans une liste

3. Vectoriser et prétraiter les données des questions

4. Vectoriser et prétraiter la requête de l'utilisateur.

5. Evaluer la question la plus similaire à la requête de l'utilisateur en utilisant la similarité cosinus

6. Renvoyer la réponse correspondante à la question la plus similaire sous forme de réponse de
chat.

<p>Le programme doit être interactif, c'est à dire il doit permettre à l'utilisateur humain anglophone et au
chatbot d'échanger continuellement jusqu'à ce que l'utilisateur demande explicitement d'arrêter. </p>
<p>Le programme doit pouvoir proposer deux types de vectorisation : le sac de mots et tf-idf.
Pour tester votre programme, établissez une liste de 6 questions parmi lesquelles 3 sont issues du
corpus questions. Le reste des questions doit être similaire aux questions de corpus sans être
exactement identiques.</p>


In [17]:
import re
import spacy
import nltk
nltk.download('stopwords')

from nltk.corpus import stopwords

stop_words = stopwords.words('english')
# Charger le modèle 'en' pour la langue anglaise
nlp = spacy.load('en_core_web_sm')

def preprocess(text, strip=True, lowercase=True, cleaning_digit_url=True, remove_stopwords=True, tokenization=True):

    if strip:
        # strip
        text = text.strip()

    if lowercase:
        # minuscule
        text = text.lower()

    if cleaning_digit_url:
        # Expression régulière pour identifier les URLs
        url_regex = r'http\S+'
        text = re.sub(url_regex, "URL", text)

        # Expression régulière pour identifier les chiffres
        digit_regex = r'\d+'
        text = re.sub(digit_regex, "DIGIT", text)

        # Expression régulière pour identifier les users
        user_regex = r'@\S+'
        text = re.sub(user_regex, "USER", text)

    if remove_stopwords:
        # suppression des mots vides
        filtered_words = [word for word in text.split() if word.lower() not in stop_words]
        text = ' '.join(filtered_words)

    if tokenization:
        # racinisation et lemmatisation
        nlp_output = nlp(text)
    
    return nlp_output.text


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Julien\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [30]:
import pandas as pd
import gzip

def parse(path):
    g = gzip.open(path, 'rb')
    for l in g:
        yield eval(l)

def getDF(path):
    i = 0
    df = {}
    for d in parse(path):
        df[i] = d
        i += 1
    return pd.DataFrame.from_dict(df, orient='index')

print("RECUPERE QUE LES 500 PREMIERS POUR DES RAISONS DE PERFORMAnCES")
df = getDF('qa_Electronics.json.gz')
questions = df['question'][:500].apply(preprocess).tolist()
answers = df['answer'][:500].tolist()

print(questions[:5])
print(answers[:5])

RECUPERE QUE LES 500 PREMIERS POUR DES RAISONS DE PERFORMAnCES
['cover one fits old nook color? believe DIGITxDIGIT.', 'fit nook glowlight?', 'would fit nook DIGITst edition? DIGIT.DIGITin x DIGIT.DIGITin ?', "fit nook color that's DIGIT x DIGIT?", 'fit samsung galaxy tab DIGIT nook DIGIT.DIGIT']
['Yes this fits both the nook color and the same-shaped nook tablet', 'No. The nook color or color tablet', "I don't think so. The nook color is 5 x 8 so not sure anything smaller would stay locked in, but would be close.", 'yes', "No, the tab is smaller than the 'color'"]


In [43]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer

def find_most_similar_sac_words(query, questions):
    vectorizer = CountVectorizer()
    vectors = vectorizer.fit_transform(questions + [query])
    
    query_vector = vectors[-1]
    question_vectors = vectors[:-1]
    
    similarity_scores = []
    for vec in question_vectors:
        score = cosine_similarity(vec, query_vector)[0][0]
        similarity_scores.append(score)
    
    most_similar_index = max(range(len(similarity_scores)), key=lambda i: similarity_scores[i])
    
    return most_similar_index

def find_most_similar(query, questions):

    vectorizer = TfidfVectorizer()
    vectors = vectorizer.fit_transform(questions + [query])
    
    similarity_scores = cosine_similarity(vectors[:-1], vectors[-1])
    
    most_similar_index = similarity_scores.argmax()
    
    return most_similar_index

def chatbot(request, questions, answers, sac_words=False):
    # 4. Vectoriser et prétraiter la requête de l'utilisateur.
    request_preprocessed = preprocess(request)

    #5. Evaluer la question la plus similaire à la requête de l'utilisateur en utilisant la similarité cosinus
    if sac_words:
        most_similar_index = find_most_similar_sac_words(request_preprocessed, questions)
    else:
        most_similar_index = find_most_similar(request_preprocessed, questions)
    
    #6. Renvoyer la réponse correspondante à la question la plus similaire sous forme de réponse de chat.
    most_similar_response = answers[most_similar_index]
    
    return most_similar_response

response = chatbot("What is the best way to clean a laptop screen?", questions, answers)
print(response)


I only use Cushion Grip in my lower plate so I let it soak in hot water and then use a paring knife to get under the pieces and pull them out. My husband uses a thicker piece in his lower denture but just grabs hold of it, pulls and it comes right out. Hope this helps!


<p>Le programme doit être interactif, c'est à dire il doit permettre à l'utilisateur humain anglophone et au
chatbot d'échanger continuellement jusqu'à ce que l'utilisateur demande explicitement d'arrêter. </p>
<p>Le programme doit pouvoir proposer deux types de vectorisation : le sac de mots et tf-idf.
Pour tester votre programme, établissez une liste de 6 questions parmi lesquelles 3 sont issues du
corpus questions. Le reste des questions doit être similaire aux questions de corpus sans être
exactement identiques.</p>


In [44]:
test_questions = [
    "Does it fit Nook GlowLight?",
    "will this fit the Samsung Galaxy Tab 4 Nook 10.1",
    "My TV has a VESA 200x200 bolt pattern and I need it to extend to 24 inches. With the extension, will this wall mount fit my bolt pattern?",

    "How does the latest macbook work?",
    "The latest macbook has how many slots?",
    "I don't have internet how can I use it?",

    "quit"
]

for question in test_questions:
    if question != "quit":
        print("User: ", question)
        response = chatbot(question, questions, answers)
        print("Bot: ", response)

def chatbot_interactive():
    question = input("User: ")
    while question != "quit":
        response = chatbot(question, questions, answers)
        print("Bot: ", response)
        question = input("User: ")

# chatbot_interactive()


User:  Does it fit Nook GlowLight?
Bot:  No. The nook color or color tablet
User:  will this fit the Samsung Galaxy Tab 4 Nook 10.1
Bot:  No, the tab is smaller than the 'color'
User:  My TV has a VESA 200x200 bolt pattern and I need it to extend to 24 inches. With the extension, will this wall mount fit my bolt pattern?
Bot:  200mm x200mm. Yes Fully extended from wall to TV support is 19 inches From wall to the front of my tv is 24 inches.
User:  How does the latest macbook work?
Bot:  I just got my MacBook Air in January, so I believe it's the latest model. It did NOT work with my computer. The vendor keeps trying to get me to take this feedback down, but I won't because it never worked from day one.
User:  The latest macbook has how many slots?
Bot:  I just got my MacBook Air in January, so I believe it's the latest model. It did NOT work with my computer. The vendor keeps trying to get me to take this feedback down, but I won't because it never worked from day one.
User:  I don't h

<p>Testez votre programme avec le sac de mots et tf-idf. Y a-t-il une différence significative en termes de
précision dans les réponses entre ces deux techniques de représentation vectorielle ?<p>

In [47]:
# Test avec TfIdf

for question in test_questions:
    if question != "quit":
        print("User: ", question)
        response = chatbot(question, questions, answers)
        print("Bot: ", response)

print("#################")

# Test avec sac de mots

for question in test_questions:
    if question != "quit":
        print("User: ", question)
        response = chatbot(question, questions, answers, sac_words=True)
        print("Bot: ", response)

User:  Does it fit Nook GlowLight?
Bot:  No. The nook color or color tablet
User:  will this fit the Samsung Galaxy Tab 4 Nook 10.1
Bot:  No, the tab is smaller than the 'color'
User:  My TV has a VESA 200x200 bolt pattern and I need it to extend to 24 inches. With the extension, will this wall mount fit my bolt pattern?
Bot:  200mm x200mm. Yes Fully extended from wall to TV support is 19 inches From wall to the front of my tv is 24 inches.
User:  How does the latest macbook work?
Bot:  I just got my MacBook Air in January, so I believe it's the latest model. It did NOT work with my computer. The vendor keeps trying to get me to take this feedback down, but I won't because it never worked from day one.
User:  The latest macbook has how many slots?
Bot:  I just got my MacBook Air in January, so I believe it's the latest model. It did NOT work with my computer. The vendor keeps trying to get me to take this feedback down, but I won't because it never worked from day one.
User:  I don't h

Les différences sont notables lorsque les phrases sortent du contexte des questions disponibles et en terme de performance également.

<p>Jusqu'à maintenant vous avez conçu un chatbot spécialisé aux produits électroniques. On s'intéresse à
généraliser ce chatbot à d'autres catégories du produit.</p>
<p>Sur le site http://jmcauley.ucsd.edu/data/amazon/qa/, téléchargez les fichiers json relatives à trois autres catégories
de produits éloignés du monde électronique. Puis adaptez votre programme à ces nouveaux fichiers.
Le test reste identique au précédent. Pour chaque catégorie, 6 questions sont à définir (3 sont issues du
corpus et les autres hors du corpus).</p>
<i>Le chatbot est-il précis dans les réponses à vos requêtes ?</i>

In [46]:
#On récupère 500 valeurs des 3 catégories questions et réponses
print("RECUPERE QUE LES 500 PREMIERS POUR DES RAISONS DE PERFORMANCES")
df = getDF('qa_Electronics.json.gz')
questions = df['question'][:500].apply(preprocess).tolist()
answers = df['answer'][:500].tolist()

df = getDF('qa_Grocery_and_Gourmet_Food.json.gz')
questions += df['question'][:500].apply(preprocess).tolist()
answers += df['answer'][:500].tolist()

df = getDF('qa_Health_and_Personal_Care.json.gz')
questions += df['question'][:500].apply(preprocess).tolist()
answers += df['answer'][:500].tolist()

print(len(questions))
print(len(answers))

RECUPERE QUE LES 500 PREMIERS POUR DES RAISONS DE PERFORMANCES
1500
1500


In [48]:
test_questions = [
    "how many batteries are actually in this product? Older reviews suggest only 4, but the image is clearly a package of four, with (pack of 2) clearly d",
    "What is the heat of this compared to the yellow and red curry?",
    "My TV has a VESA 200x200 bolt pattern and I need it to extend to 24 inches. With the extension, will this wall mount fit my bolt pattern?",

    "How does the latest macbook work?",
    "The latest macbook has how many slots?",
    "I don't have internet how can I use it?",

    "quit"
]

for question in test_questions:
    if question != "quit":
        print("User: ", question)
        response = chatbot(question, questions, answers)
        print("Bot: ", response)


User:  how many batteries are actually in this product? Older reviews suggest only 4, but the image is clearly a package of four, with (pack of 2) clearly d
Bot:  Its only one pack of 4. I bought AA. If you see another picture then whats described contact customer support. They have been having problems with having the pictures matching their descriptions
User:  What is the heat of this compared to the yellow and red curry?
Bot:  I think that the yellow is the most mild. The green has a much deeper flavor profile than the yellow and red though.
User:  My TV has a VESA 200x200 bolt pattern and I need it to extend to 24 inches. With the extension, will this wall mount fit my bolt pattern?
Bot:  200mm x200mm. Yes Fully extended from wall to TV support is 19 inches From wall to the front of my tv is 24 inches.
User:  How does the latest macbook work?
Bot:  I just got my MacBook Air in January, so I believe it's the latest model. It did NOT work with my computer. The vendor keeps trying to 

Le Chatbot est plus précis aux requêtes qui correspondent à son contexte, sinon il continue de donner des réponses assez vague, à noter que les tests utilisés ici sont réalisé avec seulement 500 entrées